# This notebook does activation of sugar GRNs on the left or individual downstream neurons in both hemispheres.

## Code in this notebook is adapted from code from Philip Shiu (see https://github.com/philshiu/Drosophila_brain_model).

### Note: This was the first version of our code that runs each simulation individually without a function and allows you to see the output of each step. The other notebooks include functions to run simulations more efficiently.

In [1]:
from model import run_exp
from model import default_params as params
import utils as utl
import pandas
from brian2 import Hz

config = {
    'path_res'  : './results/example',                              # directory to store results
    'path_comp' : './2023_03_23_completeness_630_final.csv',        # csv of the complete list of Flywire neurons
    'path_con'  : './2023_03_23_connectivity_630_final.parquet',    # connectivity data
    'n_proc'    : -1,                                               # number of CPU cores (-1: use all)
}

INFO       Cache size for target 'cython': 1301231437 MB.
You can call clear_cache('cython') to delete all files from the cache or manually delete files in the '/Users/anitadevineni/Library/Caches/cython/brian_extensions' directory. [brian2]


## Underlying connectivity data
The connectivity of the fly brain is stored in the folowing files:
- neurons present: `config['path_comp']`
- connectivity between neurons: `config['path_con]`

## Model parameters
The equation and constants for the leaky integrate and fire model are defined 
in the dictionary `default_params` in the beginning of the file `model.py`:

```
default_params = {
    # trials
    't_run'     : 1000 * ms,              # duration of trial
    'n_run'     : 30,                     # number of runs

    'v_0'       : -52 * mV,               # resting potential
    'v_rst'     : -52 * mV,               # reset potential after spike
    [...]
```
We can also change values
and pass the modified dictionary to the model (see Experiment 1).

## Define neurons

In [2]:
# Define sugar-sensing neurons in left hemisphere 
# (same ones as Shiu paper but they called it right; brain flip was discovered later)

neu_sugar_L = [
    720575940624963786,
    720575940630233916,
    720575940637568838,
    720575940638202345,
    720575940617000768,
    720575940630797113,
    720575940632889389,
    720575940621754367,
    720575940621502051,
    720575940640649691,
    720575940639332736,
    720575940616885538,
    720575940639198653,
    720575940620900446,
    720575940617937543,
    720575940632425919,
    720575940633143833,
    720575940612670570,
    720575940628853239,
    720575940629176663,
    720575940611875570,
]

Define a mapping from the flywire IDs to custom names. The above neurons are called sugar_1, sugar_2 etc:

In [3]:
flyid2name = { f: 'sugar_L_{}'.format(i+1) for i, f in enumerate(neu_sugar_L) }
flyid2name

{720575940624963786: 'sugar_L_1',
 720575940630233916: 'sugar_L_2',
 720575940637568838: 'sugar_L_3',
 720575940638202345: 'sugar_L_4',
 720575940617000768: 'sugar_L_5',
 720575940630797113: 'sugar_L_6',
 720575940632889389: 'sugar_L_7',
 720575940621754367: 'sugar_L_8',
 720575940621502051: 'sugar_L_9',
 720575940640649691: 'sugar_L_10',
 720575940639332736: 'sugar_L_11',
 720575940616885538: 'sugar_L_12',
 720575940639198653: 'sugar_L_13',
 720575940620900446: 'sugar_L_14',
 720575940617937543: 'sugar_L_15',
 720575940632425919: 'sugar_L_16',
 720575940633143833: 'sugar_L_17',
 720575940612670570: 'sugar_L_18',
 720575940628853239: 'sugar_L_19',
 720575940629176663: 'sugar_L_20',
 720575940611875570: 'sugar_L_21'}

In [4]:
# Define other neurons of interest

id_zorro_L = 720575940629888530
id_rattle_L = 720575940638103349
id_fmin_L = 720575940614763666
id_clavicle_L = 720575940655014049
id_usnea_L = 720575940632648612
id_phantom_L = 720575940616103218
id_g2n_L = 720575940620874757
id_fdg_L = 720575940631997032
id_bract1_L = 720575940645045527
id_bract2_L = 720575940610001220
id_roundup_L = 720575940623211725

id_foxglove_L = 720575940628755815
id_bluebell_L = 720575940631168057
id_mn9_L = 720575940645521262


id_zorro_R = 720575940611015122
id_rattle_R = 720575940630461660
id_fmin_R = 720575940636675214
id_clavicle_R = 720575940632648868
id_usnea_R = 720575940641366517
id_phantom_R = 720575940637763135
id_g2n_R = 720575940623718380
id_fdg_R = 720575940647030324
id_bract1_R = 720575940626557442
id_bract2_R = 720575940627285267
id_roundup_R = 720575940607272649

id_foxglove_R = 720575940611160930
id_bluebell_R = 720575940635166571
id_mn9_R = 720575940660219265

# Running simulations
## Background info:
To run a simulation exciting these nerons we have to call `run_exp` supplying the following:
- unique name for the simulation: `exp_name`
- a list of neurons we want to stimulate: `neu_sugar`
- the connectivity data: `config['path_comp']` and `config['path_con]`
- path to store the output: `config['path_res']`
- number of CPU cores use: `config['n_procs]`

The `.parquet` file created during a simulation contains all spikes events of all neurons in the model.
We load the data again from disk by passing a list of result files to the `utl.load_exps` function.

The spike times can be converted to spike rates [Hz] via `utl.get_rate`, which requires the duration of each trial.
`utl.get_rate` returns `pandas.DataFrame` objects:
1. spike rate for each neuron (rows) in each experiment (column): `df_rate`
2. standard deviation of rate across trials: `df_rate_std`

For convenience, we can optionally pass the `flyid2name` dictionary to `utl.get_rate` in order to convert flywire IDs into
meaningful names.

In [5]:
#show default params
params

{'t_run': 1. * second,
 'n_run': 30,
 'v_0': -52. * mvolt,
 'v_rst': -52. * mvolt,
 'v_th': -45. * mvolt,
 't_mbr': 20. * msecond,
 'tau': 5. * msecond,
 't_rfc': 2.2 * msecond,
 't_dly': 1.8 * msecond,
 'w_syn': 275. * uvolt,
 'r_poi': 150. * hertz,
 'r_poi2': 0. * hertz,
 'f_poi': 250,
 'eqs': '\ndv/dt = (v_0 - v + g) / t_mbr : volt (unless refractory)\ndg/dt = -g / tau               : volt (unless refractory) \nrfc                            : second\n',
 'eq_th': 'v > v_th',
 'eq_rst': 'v = v_rst; w = 0; g = 0 * mV'}

## Expt 1: Individually activate sugar GRNs (on L only) or each individual downstream neuron on both sides. Start with activation at 100 Hz.

In [6]:
# activate sugar sensing neurons at 100 Hz
params['r_poi'] = 100 * Hz
run_exp(exp_name='sugarR_100Hz', neu_exc=neu_sugar_L, params=params, **config)

>>> Skipping experiment sugarR_100Hz because results/example/sugarR_100Hz.parquet exists and force_overwrite = False


In [7]:
# Now activate each of the downstream neurons individually (on both sides)

params['r_poi'] = 100 * Hz

# neurons to activate and their names
names = ['zorro','rattle','fmin','clavicle','usnea','phantom','g2n','fdg','bract1','bract2','roundup']
ids_L = [id_zorro_L, id_rattle_L, id_fmin_L, id_clavicle_L, id_usnea_L, id_phantom_L, id_g2n_L, id_fdg_L, id_bract1_L,id_bract2_L, id_roundup_L]
ids_R = [id_zorro_R, id_rattle_R, id_fmin_R, id_clavicle_R, id_usnea_R, id_phantom_R, id_g2n_R, id_fdg_R, id_bract1_R,id_bract2_R, id_roundup_R]

for id in ids_L: # for each neuron ID on L side
    ind = ids_L.index(id) # get index
    name = names[ind] # get name
    id2 = ids_R[ind] # get ID for R side
    neurons_to_activate = [id, id2]
    run_exp(exp_name='bilat_activ_100Hz_{}'.format(name), neu_exc=neurons_to_activate, params=params, **config)

>>> Skipping experiment bilat_activ_100Hz_zorro because results/example/bilat_activ_100Hz_zorro.parquet exists and force_overwrite = False
>>> Skipping experiment bilat_activ_100Hz_rattle because results/example/bilat_activ_100Hz_rattle.parquet exists and force_overwrite = False
>>> Skipping experiment bilat_activ_100Hz_fmin because results/example/bilat_activ_100Hz_fmin.parquet exists and force_overwrite = False
>>> Skipping experiment bilat_activ_100Hz_clavicle because results/example/bilat_activ_100Hz_clavicle.parquet exists and force_overwrite = False
>>> Skipping experiment bilat_activ_100Hz_usnea because results/example/bilat_activ_100Hz_usnea.parquet exists and force_overwrite = False
>>> Skipping experiment bilat_activ_100Hz_phantom because results/example/bilat_activ_100Hz_phantom.parquet exists and force_overwrite = False
>>> Skipping experiment bilat_activ_100Hz_g2n because results/example/bilat_activ_100Hz_g2n.parquet exists and force_overwrite = False
>>> Skipping experime

In [8]:
# load activation data into a dataframe
filelist = ['./results/example/sugarR_100Hz.parquet']

for id in ids_L: 
    ind = ids_L.index(id)
    name = names[ind]
    datafilename = './results/example/bilat_activ_100Hz_' + name + '.parquet'
    filelist.append(datafilename)
    
df_spike = utl.load_exps(filelist)
df_rate_activ_100Hz, df_rate_std_activ_100Hz = utl.get_rate(df_spike, t_run=params['t_run'], n_run=params['n_run'], flyid2name=flyid2name)
df_rate_activ_100Hz

exp_name,name,bilat_activ_100Hz_bract1,bilat_activ_100Hz_bract2,bilat_activ_100Hz_clavicle,bilat_activ_100Hz_fdg,bilat_activ_100Hz_fmin,bilat_activ_100Hz_g2n,bilat_activ_100Hz_phantom,bilat_activ_100Hz_rattle,bilat_activ_100Hz_roundup,bilat_activ_100Hz_usnea,bilat_activ_100Hz_zorro,sugarR_100Hz
flyid,,,,,,,,,,,,,
720575940604735660,,NaN,NaN,0.033333,NaN,NaN,NaN,NaN,NaN,0.433333,NaN,NaN,NaN
720575940604737708,,4.366667,0.300000,0.566667,0.266667,NaN,0.033333,NaN,NaN,1.500000,NaN,NaN,NaN
720575940605494560,,0.033333,0.133333,0.266667,NaN,NaN,0.366667,NaN,NaN,7.833333,NaN,NaN,NaN
720575940605513649,,1.900000,2.300000,4.566667,0.500000,0.033333,6.866667,NaN,0.133333,17.066667,NaN,NaN,3.300000
720575940605658033,,3.800000,17.666667,12.466667,18.833333,0.300000,13.566667,NaN,0.700000,49.566667,NaN,NaN,19.066667
...,...,...,...,...,...,...,...,...,...,...,...,...,...
720575940660220289,,NaN,NaN,0.166667,NaN,NaN,0.133333,NaN,NaN,1.166667,NaN,NaN,0.466667
720575940660223873,,NaN,NaN,NaN,3.566667,NaN,NaN,NaN,0.066667,0.100000,NaN,NaN,39.333333
720575940660224641,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.133333


In [9]:
# get dataframe of rates for all neurons and all conditions

# make list of neurons whose rates to analyze - this includes 3 neurons that were not stimulated
neuron_list_L = ids_L
neuron_list_L.append(id_foxglove_L)
neuron_list_L.append(id_bluebell_L)
neuron_list_L.append(id_mn9_L)

neuron_list_R = ids_R
neuron_list_R.append(id_foxglove_R)
neuron_list_R.append(id_bluebell_R)
neuron_list_R.append(id_mn9_R)

# names of neurons in list
neuron_names = names
neuron_names.append('foxglove'),
neuron_names.append('bluebell')
neuron_names.append('MN9')

# get data
rates_100Hz_L = []
rates_std_100Hz_L = []
rates_100Hz_R = []
rates_std_100Hz_R = []

for neuron in neuron_list_L:
    rates_100Hz_L.append(df_rate_activ_100Hz.loc[neuron])
    rates_std_100Hz_L.append(df_rate_std_activ_100Hz.loc[neuron])
    
    
for neuron in neuron_list_R:
    rates_100Hz_R.append(df_rate_activ_100Hz.loc[neuron])
    rates_std_100Hz_R.append(df_rate_std_activ_100Hz.loc[neuron])
    
rates_100Hz_L = pandas.DataFrame(rates_100Hz_L) # convert to dataframe
rates_std_100Hz_L = pandas.DataFrame(rates_std_100Hz_L) # convert to dataframe
rates_100Hz_R = pandas.DataFrame(rates_100Hz_R) # convert to dataframe
rates_std_100Hz_R = pandas.DataFrame(rates_std_100Hz_R) # convert to dataframe

rates_100Hz_L = rates_100Hz_L.assign(name=neuron_names)
rates_std_100Hz_L = rates_std_100Hz_L.assign(name=neuron_names)
rates_100Hz_R = rates_100Hz_R.assign(name=neuron_names)
rates_std_100Hz_R = rates_std_100Hz_R.assign(name=neuron_names)
rates_100Hz_R

exp_name,name,bilat_activ_100Hz_bract1,bilat_activ_100Hz_bract2,bilat_activ_100Hz_clavicle,bilat_activ_100Hz_fdg,bilat_activ_100Hz_fmin,bilat_activ_100Hz_g2n,bilat_activ_100Hz_phantom,bilat_activ_100Hz_rattle,bilat_activ_100Hz_roundup,bilat_activ_100Hz_usnea,bilat_activ_100Hz_zorro,sugarR_100Hz
720575940611015122,zorro,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99.333333,8.133333
720575940630461660,rattle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,97.300000,NaN,NaN,4.833333,NaN
720575940636675214,fmin,NaN,NaN,NaN,0.033333,95.966667,NaN,NaN,NaN,NaN,NaN,NaN,NaN
720575940632648868,clavicle,NaN,NaN,99.100000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
720575940641366517,usnea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,97.033333,NaN,41.500000
720575940637763135,phantom,NaN,NaN,NaN,NaN,NaN,NaN,98.466667,NaN,NaN,NaN,NaN,NaN
720575940623718380,g2n,NaN,NaN,NaN,NaN,NaN,102.400000,NaN,NaN,NaN,NaN,NaN,NaN
720575940647030324,fdg,NaN,NaN,NaN,99.966667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.066667
720575940626557442,bract1,101.3,1.900000,0.533333,NaN,NaN,NaN,NaN,11.366667,NaN,NaN,NaN,NaN
720575940627285267,bract2,11.8,96.500000,NaN,2.733333,NaN,NaN,NaN,0.733333,NaN,NaN,NaN,NaN


In [10]:
# save dataframes to csv
df_rate_activ_100Hz.fillna(0).to_csv('results/100Hz bilat activation/100Hz_bilat_activ_allrates.csv')
df_rate_std_activ_100Hz.fillna(0).to_csv('results/100Hz bilat activation/100Hz_bilat_activ_allrates_std.csv')
rates_100Hz_L.to_csv('results/100Hz bilat activation/100Hz_bilat_activ_rates_L.csv')
rates_std_100Hz_L.to_csv('results/100Hz bilat activation/100Hz_bilat_activ_rates_L_std.csv')
rates_100Hz_R.to_csv('results/100Hz bilat activation/100Hz_bilat_activ_rates_R.csv')
rates_std_100Hz_R.to_csv('results/100Hz bilat activation/100Hz_bilat_activ_rates_R_std.csv')

# Repeat activation for 200 Hz

In [11]:
# activate sugar sensing neurons at 200 Hz
params['r_poi'] = 200 * Hz
run_exp(exp_name='sugarR_200Hz', neu_exc=neu_sugar_L, params=params, **config)

>>> Skipping experiment sugarR_200Hz because results/example/sugarR_200Hz.parquet exists and force_overwrite = False


In [12]:
# Now activate each of the downstream neurons individually but on both sides

params['r_poi'] = 200 * Hz

# neurons to activate and their names
names = ['zorro','rattle','fmin','clavicle','usnea','phantom','g2n','fdg','bract1','bract2','roundup']
ids_L = [id_zorro_L, id_rattle_L, id_fmin_L, id_clavicle_L, id_usnea_L, id_phantom_L, id_g2n_L, id_fdg_L, id_bract1_L,id_bract2_L, id_roundup_L]
ids_R = [id_zorro_R, id_rattle_R, id_fmin_R, id_clavicle_R, id_usnea_R, id_phantom_R, id_g2n_R, id_fdg_R, id_bract1_R,id_bract2_R, id_roundup_R]

for id in ids_L: # for each neuron ID on L side
    ind = ids_L.index(id) # get index
    name = names[ind] # get name
    id2 = ids_R[ind] # get ID for R side
    neurons_to_activate = [id, id2]
    run_exp(exp_name='bilat_activ_200Hz_{}'.format(name), neu_exc=neurons_to_activate, params=params, **config)

>>> Skipping experiment bilat_activ_200Hz_zorro because results/example/bilat_activ_200Hz_zorro.parquet exists and force_overwrite = False
>>> Skipping experiment bilat_activ_200Hz_rattle because results/example/bilat_activ_200Hz_rattle.parquet exists and force_overwrite = False
>>> Skipping experiment bilat_activ_200Hz_fmin because results/example/bilat_activ_200Hz_fmin.parquet exists and force_overwrite = False
>>> Skipping experiment bilat_activ_200Hz_clavicle because results/example/bilat_activ_200Hz_clavicle.parquet exists and force_overwrite = False
>>> Skipping experiment bilat_activ_200Hz_usnea because results/example/bilat_activ_200Hz_usnea.parquet exists and force_overwrite = False
>>> Skipping experiment bilat_activ_200Hz_phantom because results/example/bilat_activ_200Hz_phantom.parquet exists and force_overwrite = False
>>> Skipping experiment bilat_activ_200Hz_g2n because results/example/bilat_activ_200Hz_g2n.parquet exists and force_overwrite = False
>>> Skipping experime

In [13]:
# load activation data into a dataframe

filelist = ['./results/example/sugarR_200Hz.parquet']

for id in ids_L: 
    ind = ids_L.index(id)
    name = names[ind]
    datafilename = './results/example/bilat_activ_200Hz_' + name + '.parquet'
    filelist.append(datafilename)
    
df_spike = utl.load_exps(filelist)
df_rate_activ_200Hz, df_rate_std_activ_200Hz = utl.get_rate(df_spike, t_run=params['t_run'], n_run=params['n_run'], flyid2name=flyid2name)
df_rate_activ_200Hz

exp_name,name,bilat_activ_200Hz_bract1,bilat_activ_200Hz_bract2,bilat_activ_200Hz_clavicle,bilat_activ_200Hz_fdg,bilat_activ_200Hz_fmin,bilat_activ_200Hz_g2n,bilat_activ_200Hz_phantom,bilat_activ_200Hz_rattle,bilat_activ_200Hz_roundup,bilat_activ_200Hz_usnea,bilat_activ_200Hz_zorro,sugarR_200Hz
flyid,,,,,,,,,,,,,
720575940604735660,,0.200000,NaN,0.100000,NaN,NaN,NaN,NaN,NaN,0.300000,NaN,NaN,NaN
720575940604737708,,15.666667,3.333333,4.866667,0.033333,NaN,0.233333,NaN,0.166667,3.100000,NaN,NaN,NaN
720575940605420902,,NaN,NaN,NaN,0.033333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.600000
720575940605494560,,0.066667,0.400000,1.600000,NaN,NaN,1.300000,NaN,NaN,11.266667,NaN,NaN,NaN
720575940605513649,,9.800000,5.800000,12.933333,0.633333,1.966667,14.600000,NaN,4.166667,26.433333,NaN,0.433333,5.933333
...,...,...,...,...,...,...,...,...,...,...,...,...,...
720575940660223873,,NaN,NaN,NaN,5.166667,NaN,NaN,NaN,3.233333,5.166667,NaN,NaN,62.933333
720575940660224641,,NaN,NaN,0.066667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.066667
720575940660229505,,11.700000,6.266667,13.866667,20.100000,28.066667,5.466667,NaN,7.666667,51.866667,NaN,0.033333,46.666667


In [14]:
# get dataframe of rates for all neurons and all conditions

# make list of neurons whose rates to analyze - this includes 3 neurons that were not stimulated
neuron_list_L = ids_L
neuron_list_L.append(id_foxglove_L)
neuron_list_L.append(id_bluebell_L)
neuron_list_L.append(id_mn9_L)

neuron_list_R = ids_R
neuron_list_R.append(id_foxglove_R)
neuron_list_R.append(id_bluebell_R)
neuron_list_R.append(id_mn9_R)

# names of neurons in list
neuron_names = names
neuron_names.append('foxglove'),
neuron_names.append('bluebell')
neuron_names.append('MN9')

# get data
rates_200Hz_L = []
rates_std_200Hz_L = []
rates_200Hz_R = []
rates_std_200Hz_R = []

for neuron in neuron_list_L:
    rates_200Hz_L.append(df_rate_activ_200Hz.loc[neuron])
    rates_std_200Hz_L.append(df_rate_std_activ_200Hz.loc[neuron])
    
for neuron in neuron_list_R:
    rates_200Hz_R.append(df_rate_activ_200Hz.loc[neuron])
    rates_std_200Hz_R.append(df_rate_std_activ_200Hz.loc[neuron])
    
rates_200Hz_L = pandas.DataFrame(rates_200Hz_L) # convert to dataframe
rates_std_200Hz_L = pandas.DataFrame(rates_std_200Hz_L) # convert to dataframe
rates_200Hz_R = pandas.DataFrame(rates_200Hz_R) # convert to dataframe
rates_std_200Hz_R = pandas.DataFrame(rates_std_200Hz_R) # convert to dataframe

rates_200Hz_L = rates_200Hz_L.assign(name=neuron_names)
rates_std_200Hz_L = rates_std_200Hz_L.assign(name=neuron_names)
rates_200Hz_R = rates_200Hz_R.assign(name=neuron_names)
rates_std_200Hz_R = rates_std_200Hz_R.assign(name=neuron_names)
rates_200Hz_R

exp_name,name,bilat_activ_200Hz_bract1,bilat_activ_200Hz_bract2,bilat_activ_200Hz_clavicle,bilat_activ_200Hz_fdg,bilat_activ_200Hz_fmin,bilat_activ_200Hz_g2n,bilat_activ_200Hz_phantom,bilat_activ_200Hz_rattle,bilat_activ_200Hz_roundup,bilat_activ_200Hz_usnea,bilat_activ_200Hz_zorro,sugarR_200Hz
720575940611015122,zorro,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,197.300000,49.466667
720575940630461660,rattle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,195.966667,NaN,NaN,21.833333,NaN
720575940636675214,fmin,NaN,NaN,NaN,0.066667,196.366667,NaN,NaN,NaN,NaN,NaN,NaN,NaN
720575940632648868,clavicle,NaN,NaN,193.2,NaN,NaN,0.066667,NaN,NaN,NaN,NaN,NaN,NaN
720575940641366517,usnea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,194.4,NaN,75.600000
720575940637763135,phantom,NaN,NaN,NaN,NaN,NaN,NaN,197.033333,NaN,NaN,NaN,NaN,NaN
720575940623718380,g2n,NaN,NaN,NaN,NaN,NaN,195.400000,NaN,NaN,NaN,NaN,NaN,0.066667
720575940647030324,fdg,NaN,NaN,NaN,195.866667,NaN,0.033333,NaN,6.600000,NaN,NaN,NaN,17.466667
720575940626557442,bract1,198.500000,4.933333,0.4,NaN,NaN,NaN,NaN,21.900000,NaN,NaN,NaN,NaN
720575940627285267,bract2,27.600000,197.133333,NaN,13.800000,NaN,NaN,NaN,4.000000,NaN,NaN,NaN,NaN


In [15]:
# save dataframes to csv
df_rate_activ_200Hz.fillna(0).to_csv('results/200Hz bilat activation/200Hz_bilat_activ_allrates.csv')
df_rate_std_activ_200Hz.fillna(0).to_csv('results/200Hz bilat activation/200Hz_bilat_activ_allrates_std.csv')
rates_200Hz_L.to_csv('results/200Hz bilat activation/200Hz_bilat_activ_rates_L.csv')
rates_std_200Hz_L.to_csv('results/200Hz bilat activation/200Hz_bilat_activ_rates_L_std.csv')
rates_200Hz_R.to_csv('results/200Hz bilat activation/200Hz_bilat_activ_rates_R.csv')
rates_std_200Hz_R.to_csv('results/200Hz bilat activation/200Hz_bilat_activ_rates_R_std.csv')

# Repeat activation for 50 Hz

In [16]:
# activate sugar sensing neurons at 50 Hz
params['r_poi'] = 50 * Hz
run_exp(exp_name='sugarR_50Hz', neu_exc=neu_sugar_L, params=params, **config)

>>> Skipping experiment sugarR_50Hz because results/example/sugarR_50Hz.parquet exists and force_overwrite = False


In [17]:
# Now activate each of the downstream neurons individually but on both sides

params['r_poi'] = 50 * Hz

# neurons to activate and their names
names = ['zorro','rattle','fmin','clavicle','usnea','phantom','g2n','fdg','bract1','bract2','roundup']
ids_L = [id_zorro_L, id_rattle_L, id_fmin_L, id_clavicle_L, id_usnea_L, id_phantom_L, id_g2n_L, id_fdg_L, id_bract1_L,id_bract2_L, id_roundup_L]
ids_R = [id_zorro_R, id_rattle_R, id_fmin_R, id_clavicle_R, id_usnea_R, id_phantom_R, id_g2n_R, id_fdg_R, id_bract1_R,id_bract2_R, id_roundup_R]

for id in ids_L: # for each neuron ID on L side
    ind = ids_L.index(id) # get index
    name = names[ind] # get name
    id2 = ids_R[ind] # get ID for R side
    neurons_to_activate = [id, id2]
    run_exp(exp_name='bilat_activ_50Hz_{}'.format(name), neu_exc=neurons_to_activate, params=params, **config)

>>> Skipping experiment bilat_activ_50Hz_zorro because results/example/bilat_activ_50Hz_zorro.parquet exists and force_overwrite = False
>>> Skipping experiment bilat_activ_50Hz_rattle because results/example/bilat_activ_50Hz_rattle.parquet exists and force_overwrite = False
>>> Skipping experiment bilat_activ_50Hz_fmin because results/example/bilat_activ_50Hz_fmin.parquet exists and force_overwrite = False
>>> Skipping experiment bilat_activ_50Hz_clavicle because results/example/bilat_activ_50Hz_clavicle.parquet exists and force_overwrite = False
>>> Skipping experiment bilat_activ_50Hz_usnea because results/example/bilat_activ_50Hz_usnea.parquet exists and force_overwrite = False
>>> Skipping experiment bilat_activ_50Hz_phantom because results/example/bilat_activ_50Hz_phantom.parquet exists and force_overwrite = False
>>> Skipping experiment bilat_activ_50Hz_g2n because results/example/bilat_activ_50Hz_g2n.parquet exists and force_overwrite = False
>>> Skipping experiment bilat_activ

In [18]:
# load activation data into a dataframe

filelist = ['./results/example/sugarR_50Hz.parquet']

for id in ids_L: 
    ind = ids_L.index(id)
    name = names[ind]
    datafilename = './results/example/bilat_activ_50Hz_' + name + '.parquet'
    filelist.append(datafilename)
    
df_spike = utl.load_exps(filelist)
df_rate_activ_50Hz, df_rate_std_activ_50Hz = utl.get_rate(df_spike, t_run=params['t_run'], n_run=params['n_run'], flyid2name=flyid2name)
df_rate_activ_50Hz

exp_name,name,bilat_activ_50Hz_bract1,bilat_activ_50Hz_bract2,bilat_activ_50Hz_clavicle,bilat_activ_50Hz_fdg,bilat_activ_50Hz_fmin,bilat_activ_50Hz_g2n,bilat_activ_50Hz_phantom,bilat_activ_50Hz_rattle,bilat_activ_50Hz_roundup,bilat_activ_50Hz_usnea,bilat_activ_50Hz_zorro,sugarR_50Hz
flyid,,,,,,,,,,,,,
720575940604735660,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.233333,NaN,NaN,NaN
720575940604737708,,0.300000,0.033333,NaN,0.033333,NaN,NaN,NaN,NaN,1.100000,NaN,NaN,NaN
720575940605494560,,NaN,NaN,NaN,NaN,NaN,0.100000,NaN,NaN,2.866667,NaN,NaN,NaN
720575940605513649,,0.166667,0.433333,0.200000,0.266667,NaN,0.966667,NaN,NaN,9.566667,NaN,NaN,0.600000
720575940605658033,,0.466667,4.700000,1.366667,6.933333,NaN,3.366667,NaN,0.100000,28.900000,NaN,NaN,3.900000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
720575940660219265,,2.866667,25.333333,10.200000,21.000000,0.500000,12.700000,NaN,2.633333,76.933333,NaN,NaN,18.866667
720575940660219521,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.033333
720575940660223873,,NaN,NaN,NaN,1.133333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.133333


In [19]:
# get dataframe of rates for all neurons and all conditions

# make list of neurons whose rates to analyze - this includes 3 neurons that were not stimulated
neuron_list_L = ids_L
neuron_list_L.append(id_foxglove_L)
neuron_list_L.append(id_bluebell_L)
neuron_list_L.append(id_mn9_L)

neuron_list_R = ids_R
neuron_list_R.append(id_foxglove_R)
neuron_list_R.append(id_bluebell_R)
neuron_list_R.append(id_mn9_R)

# names of neurons in list
neuron_names = names
neuron_names.append('foxglove'),
neuron_names.append('bluebell')
neuron_names.append('MN9')

# get data
rates_50Hz_L = []
rates_std_50Hz_L = []
rates_50Hz_R = []
rates_std_50Hz_R = []

for neuron in neuron_list_L:
    rates_50Hz_L.append(df_rate_activ_50Hz.loc[neuron])
    rates_std_50Hz_L.append(df_rate_std_activ_50Hz.loc[neuron])
    
for neuron in neuron_list_R:
    rates_50Hz_R.append(df_rate_activ_50Hz.loc[neuron])
    rates_std_50Hz_R.append(df_rate_std_activ_50Hz.loc[neuron])
    
rates_50Hz_L = pandas.DataFrame(rates_50Hz_L) # convert to dataframe
rates_std_50Hz_L = pandas.DataFrame(rates_std_50Hz_L) # convert to dataframe
rates_50Hz_R = pandas.DataFrame(rates_50Hz_R) # convert to dataframe
rates_std_50Hz_R = pandas.DataFrame(rates_std_50Hz_R) # convert to dataframe

rates_50Hz_L = rates_50Hz_L.assign(name=neuron_names)
rates_std_50Hz_L = rates_std_50Hz_L.assign(name=neuron_names)
rates_50Hz_R = rates_50Hz_R.assign(name=neuron_names)
rates_std_50Hz_R = rates_std_50Hz_R.assign(name=neuron_names)
rates_50Hz_R

exp_name,name,bilat_activ_50Hz_bract1,bilat_activ_50Hz_bract2,bilat_activ_50Hz_clavicle,bilat_activ_50Hz_fdg,bilat_activ_50Hz_fmin,bilat_activ_50Hz_g2n,bilat_activ_50Hz_phantom,bilat_activ_50Hz_rattle,bilat_activ_50Hz_roundup,bilat_activ_50Hz_usnea,bilat_activ_50Hz_zorro,sugarR_50Hz
720575940611015122,zorro,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.900000,0.200000
720575940630461660,rattle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49.966667,NaN,NaN,0.066667,NaN
720575940636675214,fmin,NaN,NaN,NaN,NaN,48.800000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
720575940632648868,clavicle,NaN,NaN,50.166667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
720575940641366517,usnea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.3,NaN,10.700000
720575940637763135,phantom,NaN,NaN,NaN,NaN,NaN,NaN,48.733333,NaN,NaN,NaN,NaN,NaN
720575940623718380,g2n,NaN,NaN,NaN,NaN,NaN,52.200000,NaN,NaN,NaN,NaN,NaN,NaN
720575940647030324,fdg,NaN,NaN,NaN,50.933333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.633333
720575940626557442,bract1,50.933333,0.233333,NaN,NaN,NaN,NaN,NaN,8.166667,NaN,NaN,NaN,NaN
720575940627285267,bract2,2.866667,49.300000,NaN,0.466667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
# save dataframes to csv
df_rate_activ_50Hz.fillna(0).to_csv('results/50Hz bilat activation/50Hz_bilat_activ_allrates.csv')
df_rate_std_activ_50Hz.fillna(0).to_csv('results/50Hz bilat activation/50Hz_bilat_activ_allrates_std.csv')
rates_50Hz_L.to_csv('results/50Hz bilat activation/50Hz_bilat_activ_rates_L.csv')
rates_std_50Hz_L.to_csv('results/50Hz bilat activation/50Hz_bilat_activ_rates_L_std.csv')
rates_50Hz_R.to_csv('results/50Hz bilat activation/50Hz_bilat_activ_rates_R.csv')
rates_std_50Hz_R.to_csv('results/50Hz bilat activation/50Hz_bilat_activ_rates_R_std.csv')